In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from pathlib import Path
import pandas as pd
import requests
import io
import zipfile

In [3]:
def fetch_raw_data(year: int, month: int) -> str:
    """
    Fetches the raw data for the specified year and month from the given URL,
    extracts CSV files from the ZIP, merges them if multiple, and saves as a single CSV.

    Args:
        year (int): The year of the data to fetch.
        month (int): The month of the data to fetch.

    Returns:
        str: The path to the merged CSV file.
    """
    url = f'https://s3.amazonaws.com/tripdata/{year}{month:02}-citibike-tripdata.csv.zip'

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f'{url} is not available')

    raw_data_dir = Path('..') / 'data' / 'raw'
    raw_data_dir.mkdir(parents=True, exist_ok=True)

    # Path to save the combined data as parquet
    output_file_path = raw_data_dir / f'citi_bike_rides_{year}_{month:02}.parquet'

    try:
        expected_columns = ['ride_id',
                            'rideable_type',
                            'started_at',
                            'ended_at',
                            'start_station_name',
                            'start_station_id',
                            'end_station_name',
                            'end_station_id',
                            'start_lat',
                            'start_lng',
                            'end_lat',
                            'end_lng',
                            'member_casual']
        
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            csv_files = [file for file in zip_file.namelist() if file.endswith('.csv')]

            if not csv_files:
                raise Exception(f'No CSV files found in {url}')

            dataframes = []
            for file in csv_files:
                with zip_file.open(file) as file:
                    data_df = pd.read_csv(file, encoding = 'latin1', on_bad_lines = 'skip')
                    data_df = data_df[[col for col in expected_columns if col in data_df.columns]]
                    dataframes.append(data_df)

            merged_df = pd.concat(dataframes, ignore_index=True)

            # Drop NaNs if exist
            merged_df = merged_df.dropna()

            # Clean the data: Convert start_station_id and end_station_id to strings
            merged_df['start_station_id'] = merged_df['start_station_id'].astype(str)
            merged_df['end_station_id'] = merged_df['end_station_id'].astype(str)

            # Convert started_at and ended_at to datetime
            merged_df['started_at'] = pd.to_datetime(merged_df['started_at'], errors='coerce')
            merged_df['ended_at'] = pd.to_datetime(merged_df['ended_at'], errors='coerce')

            # Ensure other columns are of appropriate types
            merged_df['start_lat'] = merged_df['start_lat'].astype(float, errors='ignore')
            merged_df['start_lng'] = merged_df['start_lng'].astype(float, errors='ignore')
            merged_df['end_lat'] = merged_df['end_lat'].astype(float, errors='ignore')
            merged_df['end_lng'] = merged_df['end_lng'].astype(float, errors='ignore')
            merged_df['member_casual'] = merged_df['member_casual'].astype(str)

            # Inspect the DataFrame after cleaning
            print('\nDataFrame info after cleaning:')
            print(merged_df.info())

            # Save the combined DataFrame as a parquet file
            merged_df.to_parquet(output_file_path, engine = 'pyarrow', index = False)
            print(f'Successfully fetched and saved: {str(output_file_path)}')
            return str(output_file_path)

    except Exception as e:
        raise Exception(f'Error processing data from {url}: {str(e)}')

In [4]:
fetch_raw_data(2024, 1)

/var/folders/10/l7cxtpss5ygbsz2r9p4pc_jw0000gn/T/ipykernel_21665/420891319.py:49: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(file, encoding = 'latin1', on_bad_lines = 'skip')



DataFrame info after cleaning:
<class 'pandas.core.frame.DataFrame'>
Index: 1881977 entries, 0 to 1888084
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 201.0+ MB
None
Successfully fetched and saved: ../data/raw/citi_bike_rides_2024_01.parquet


'../data/raw/citi_bike_rides_2024_01.parquet'